In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import concurrent.futures

# ---------------------------------------------
# 1. Generación de datos sintéticos
# ---------------------------------------------
def generar_datos_sinteticos(num_muestras=5000):
    """
    Genera un conjunto de datos sintético.
    Cada muestra tiene 10 características y la etiqueta es binaria,
    definida de forma simple en función de la suma de las características.
    """
    X = np.random.randn(num_muestras, 10)  # Genera datos aleatorios con distribución normal
    y = (np.sum(X, axis=1) > 0).astype(int)  # Etiqueta: 1 si la suma es positiva, 0 en otro caso
    return X, y

# ---------------------------------------------
# 2. Definición de la red neuronal simple
# ---------------------------------------------
class RedNeuronalSimple(nn.Module):
    def __init__(self, dim_entrada=10, dim_oculta=32, dim_salida=2):
        super(RedNeuronalSimple, self).__init__()
        # Definición de la arquitectura de la red neuronal
        self.modelo = nn.Sequential(
            nn.Linear(dim_entrada, dim_oculta),  # Capa de entrada a capa oculta
            nn.ReLU(),  # Función de activación ReLU
            nn.Linear(dim_oculta, dim_salida)  # Capa oculta a capa de salida
        )
    
    def forward(self, x):
        return self.modelo(x)  # Propagación hacia adelante

# ---------------------------------------------
# 3. Función de entrenamiento con DataLoader optimizado
# ---------------------------------------------
def entrenar_modelo(modelo, X_train, y_train, epochs=5, batch_size=128):
    """
    Entrena el modelo usando Adam y CrossEntropyLoss.
    Se utiliza DataLoader con 'num_workers' > 0 para paralelizar la carga de datos.
    """
    optimizer = optim.Adam(modelo.parameters(), lr=0.001)  # Optimizador Adam
    criterio = nn.CrossEntropyLoss()  # Función de pérdida de entropía cruzada
    modelo.train()  # Establecer el modelo en modo de entrenamiento
    
    # Convertir a tensores y crear un dataset
    tensor_X = torch.tensor(X_train, dtype=torch.float32)  # Convertir características a tensor
    tensor_y = torch.tensor(y_train, dtype=torch.long)  # Convertir etiquetas a tensor
    dataset = torch.utils.data.TensorDataset(tensor_X, tensor_y)  # Crear dataset
    
    # Uso de DataLoader con 'num_workers' para paralelizar la carga de datos
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    for epoch in range(epochs):
        perdida_epoch = 0.0  # Inicializar pérdida de la época
        for batch_X, batch_y in dataloader:  # Iterar sobre los batches
            optimizer.zero_grad()  # Reiniciar los gradientes
            salidas = modelo(batch_X)  # Obtener las salidas del modelo
            perdida = criterio(salidas, batch_y)  # Calcular la pérdida
            perdida.backward()  # Retropropagación
            optimizer.step()  # Actualizar los parámetros
            perdida_epoch += perdida.item()  # Acumular pérdida de la época
        print(f"Epoch {epoch+1}/{epochs}, Pérdida Promedio: {perdida_epoch/len(dataloader):.4f}")

# ---------------------------------------------
# 4. Funciones para inferencia en paralelo
# ---------------------------------------------
def predecir(modelo, X_data):
    """
    Realiza la inferencia sobre un bloque de datos.
    Se utiliza 'torch.no_grad()' para desactivar el cálculo de gradientes.
    """
    modelo.eval()  # Establecer el modelo en modo de evaluación
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        tensor_X = torch.tensor(X_data, dtype=torch.float32)  # Convertir datos a tensor
        salidas = modelo(tensor_X)  # Obtener las salidas del modelo
        # Seleccionar la clase con mayor probabilidad
        predicciones = torch.argmax(salidas, dim=1).numpy()  # Obtener predicciones
    return predicciones

def inferencia_paralela(modelo, X_data, num_hilos=4):
    """
    Divide el conjunto de datos en 'num_hilos' partes y realiza la inferencia en paralelo.
    Se utiliza ThreadPoolExecutor ya que las operaciones en PyTorch liberan el GIL.
    """
    # Dividir los datos en chunks
    total = len(X_data)  # Total de datos
    tam_chunk = total // num_hilos  # Tamaño de cada chunk
    chunks = [X_data[i*tam_chunk:(i+1)*tam_chunk] for i in range(num_hilos)]  # Crear chunks
    
    # Si no es divisible exactamente, agregar los datos sobrantes al último chunk
    if total % num_hilos != 0:
        chunks[-1] = np.concatenate((chunks[-1], X_data[num_hilos*tam_chunk:]), axis=0)  # Agregar sobrantes
    
    # Uso de ThreadPoolExecutor para paralelizar la inferencia
    resultados = [None] * num_hilos  # Inicializar lista de resultados
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_hilos) as executor:
        # Se asocia cada chunk con su índice para mantener el orden
        futures = {executor.submit(predecir, modelo, chunk): idx for idx, chunk in enumerate(chunks)}
        for future in concurrent.futures.as_completed(futures):  # Esperar a que se completen las tareas
            idx = futures[future]  # Obtener el índice del chunk
            resultados[idx] = future.result()  # Almacenar el resultado
    
    # Concatenar los resultados y devolver
    return np.concatenate(resultados)  # Devolver todas las predicciones

# ---------------------------------------------
# 5. Función principal
# ---------------------------------------------
def main():
    # Generar datos sintéticos y dividir en entrenamiento y prueba
    X, y = generar_datos_sinteticos(num_muestras=5000)  # Generar datos
    indice_split = int(0.8 * len(X))  # Índice para dividir los datos
    X_train, X_test = X[:indice_split], X[indice_split:]  # Dividir en entrenamiento y prueba
    y_train, y_test = y[:indice_split], y[indice_split:]  # Dividir etiquetas
    
    # Inicializar el modelo
    modelo = RedNeuronalSimple(dim_entrada=10, dim_oculta=32, dim_salida=2)  # Crear modelo
    
    # Entrenamiento del modelo
    print("Iniciando el entrenamiento del modelo...")
    inicio = time.time()  # Iniciar temporizador
    entrenar_modelo(modelo, X_train, y_train, epochs=5, batch_size=128)  # Entrenar modelo
    print(f"Entrenamiento completado en {time.time() - inicio:.2f} segundos.\n")  # Mostrar tiempo de entrenamiento
    
    # Inferencia en paralelo sobre el conjunto de prueba
    print("Realizando inferencia en paralelo sobre el conjunto de prueba...")
    inicio = time.time()  # Iniciar temporizador
    predicciones = inferencia_paralela(modelo, X_test, num_hilos=4)  # Realizar inferencia
    print(f"Inferencia completada en {time.time() - inicio:.2f} segundos.")  # Mostrar tiempo de inferencia
    
    # Evaluación de la precisión
    precision = np.mean(predicciones == y_test)  # Calcular precisión
    print(f"Precisión en Test: {precision*100:.2f}%")  # Mostrar precisión
    
if __name__ == "__main__":
    main()  # Ejecutar función principal